# Notebook 20: Singular Value Decomposition
***

In this notebook, we will decompose a delicious and scientific matrix using singular value decomposition, and perform dimension reduction by discarding the components that have low importance, as measured by their singular values.

We'll need numpy for this notebook, so let's load it.

In [1]:
import numpy as np

<br>

### Exercise 1: Computing SVD

Suppose you have developed technology to reanimate dead scientists and survey their modern-day cereal preferences, where a rating of 0 indicates they have not tried the cereal, 1 is a bad rating and 5 is a good one. You obtain the following data matrix:

$$\begin{array}{c|ccc}
                   & \text{Wheaties} & \text{Lucky Charms} & \text{Cookie Crisp} & \text{Weetabix} \\
   \hline
 \text{Ampere}       & 5 & 1 & 0 & 3 \\
 \text{Bernoulli}    & 2 & 2 & 5 & 1 \\
 \text{Curie}        & 5 & 0 & 0 & 4 \\
 \text{Darwin}       & 0 & 4 & 3 & 0 \\
 \text{Eratosthenes} & 0 & 2 & 3 & 0 \\
 \text{Fisher}       & 4 & 0 & 0 & 3 \\
 \end{array}$$

In [2]:
M = np.array([[5,1,0,3],
              [2,2,5,1],
              [5,0,0,4],
              [0,4,3,0],
              [0,2,3,0],
              [4,0,0,3]])

We'll get to computing SVD by hand using the methods from lecture in a moment, but first it would be nice to have a method for checking whether our future calculation is correct or not. Numpy to the rescue! Specifically, `numpy.linalg.svd` will take a matrix $M$ we wish to decompose using SVD, and return the $U$, $\Sigma$ and $V^T$ matrices (more or less), such that $M = U\Sigma V^T$. Check out the documentation for [numpy.linalg.svd](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html) for more details on the return values.

In [3]:
U, Sig, VT = np.linalg.svd(M)
print("Singular values are: ", np.round(Sig, 4))

Singular values are:  [10.4609  7.6557  2.1351  0.6334]


Let's look at the shapes of the three arrays that are returned:

In [4]:
print(U.shape)
print(Sig.shape)
print(VT.shape)

(6, 6)
(4,)
(4, 4)


Notice that in class, the $U$ matrix would have had shape $6 \times 4$, but here we have $6\times 6$. What's up with that?

Recall that the columns of $U$ are the eigenvectors of $MM^T$. What size is that matrix, and how many eigenvectors should it have?

**Solution:**

$M$ is $6 \times 4$, so $MM^T$ is $6 \times 6$. So, it should have 6 eigenpairs. Thus, $U$ is of the proper size.

In order to recover the original matrix $M$ and make sure our SVD is working properly, we would like to multiply $U \Sigma V^T$, but the sizes aren't right. First off, $\Sigma$ should actually be a diagonal matrix with the singular values down the diagonal.

In [5]:
Sig = np.diag(Sig)
print(Sig)

[[10.46087701  0.          0.          0.        ]
 [ 0.          7.65572277  0.          0.        ]
 [ 0.          0.          2.13513866  0.        ]
 [ 0.          0.          0.          0.63335925]]


Multiplying $\Sigma$ on the right by $V^T$ is valid, since the number of columns of $\Sigma$ is 4, which matches the number of rows in $V^T$. But multiplying on the left by $U$ won't work since the number of rows in $\Sigma$ is only 4, whereas $U$ contains all 6 eigenvectors of $MM^T$. We can remedy this by padding $\Sigma$ with extra rows of $0$s to match the number of columns in $U$.

In [8]:
# full Sigma matrix should match U's columns and V's rows
Sig_full = np.zeros((U.shape[1], VT.shape[0]))

# fill in the singular values
Sig_full[:Sig.shape[0], :Sig.shape[1]] = Sig

print(Sig_full)

[[10.46087701  0.          0.          0.        ]
 [ 0.          7.65572277  0.          0.        ]
 [ 0.          0.          2.13513866  0.        ]
 [ 0.          0.          0.          0.63335925]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]


Now compute the matrix product $U \Sigma V^T$, using our padded $\Sigma$ matrix and the output from the `np.linalg.svd` function call. How well does it match the original $M$ matrix? Eyeballing this is difficult, so you may want to use a popular matrix norm.

In [9]:
# SOLUTION:

# we compute the frobenius norm (default for np.linalg.norm)
print(np.linalg.norm(M - np.matmul(np.matmul(U, Sig_full), VT)))

1.1639882322073053e-14


<br>

### Exercise 2: A Journey into Concept Space

Recall that a singular value decomposition can be thought of as rotating our original data into ***concept space*** and performing dimension reduction by only retaining the most important concepts (i.e., the highest singular values). Based on the original data set, what do you think the concepts are? If you aren't sure, try simply performing a Google Image Search for the four cereals in the data set.

Now, rotate the original data matrix $M$ into concept space by multiplying on the right by $V$. Discuss with those around you the cereal preferences of the scientists. Which scientist(s) display strong preference for one type of cereal over another? Which scientist(s) are not as picky? Is it ironic that any of these scientists does not seem to display a binary love-it-or-hate-it relationship with types of cereal?

In [10]:
V = np.transpose(VT)
MV = np.matmul(M, V)
print(MV)

[[-5.72658974 -1.23291163 -0.69463608  0.45119793]
 [-3.69733755  4.30012552  1.34825993  0.14425925]
 [-6.07450887 -1.98941548  0.10929952 -0.36141622]
 [-1.50244804  4.52833007 -1.49042764 -0.12450749]
 [-1.1121463   3.42507916  0.13145117 -0.12117721]
 [-4.75099309 -1.55055671  0.08884038 -0.12627712]]


Each of the columns of the matrix $MV$ corresponds to the scientists' preferences along each of the concepts. Examine the magnitudes (don't worry about positive or negative values) for each concept. Which concept(s) do you hypothesize are important to retain in our reduced-rank representation of the data set, and which concept(s) do you think we can safely discard? Why?

**Solution:**

The first two concepts have strengths greater than 1, so we should probably keep those. The last concept has small strengths, all less than 1, so we can probably safely get rid of that one. The third concept is a bit of a toss-up, so we should compute the energy (sum of singular values squared) to figure out whether or not to keep it!

<br>

### Exercise 3: Reducing Dimensionality

Compute the proportion of the total energy ($\sum \sigma_k^2$, where $\sigma_k$ is the $k^{th}$ singular value) that is retained by eliminating the 1, 2 or 3 lowest singular values. As a general guideline, when we reduce dimensionality using SVD, we aim to keep about 80-90% of the total energy.

In [11]:
total_energy = 0 # TODO -- sum the squares of all singular values
frac_energy_retained = np.zeros(len(Sig)) # initialize
for k in range(1,len(frac_energy_retained)+1):
    frac_energy_retained[k-1] = 0 # TODO - ratio of (sum of k largest singular values) to total energy

print(frac_energy_retained)

[0. 0. 0. 0.]


In [12]:
# SOLUTION:

total_energy = np.sum(Sig**2)
frac_energy_retained = np.zeros(len(Sig))
for k in range(1,len(frac_energy_retained)+1):
    frac_energy_retained[k-1] = np.sum(Sig[:k,:k]**2)/total_energy
    
print(frac_energy_retained)

[0.63254305 0.97132971 0.99768125 1.        ]


Which components should we keep in our reduced rank representation?

**Solution:**

The components along the first two singular vectors account for 97% of the energy, so we should be safe to discard the last two components.

Compute the reduced rank approximation of $M$ by zero-ing out the columns of $U$, rows of $V^T$, and elements of $\Sigma$ associated with the unimportant concepts. What is the distance between the reduced rank matrix and the original data matrix, using the Frobenius norm?

In [ ]:
keep = 0 # TODO! decide how many to keep based on the energy
U_red = U[:,:keep]
Sig_red = Sig_full[:keep,:keep]
VT_red = VT[:keep,:]
M_red = np.matmul(np.matmul(U_red, Sig_red), VT_red)
print(M_red)

print(np.linalg.norm(M_red-M))

In [13]:
# SOLUTION:

keep = 2
U_red = U[:,:keep]
Sig_red = Sig_full[:keep,:keep]
VT_red = VT[:keep,:]
M_red = np.matmul(np.matmul(U_red, Sig_red), VT_red)
print(M_red)

print(np.linalg.norm(M_red-M))

[[ 4.7577699   0.43744354  0.42369952  3.36253603]
 [ 1.87942647  3.09359732  4.21768531  1.12691028]
 [ 5.20630897  0.08803348 -0.07807664  3.70647177]
 [ 0.11303884  2.79114616  3.86617955 -0.11182255]
 [ 0.06656856  2.10639716  2.91841412 -0.09775144]
 [ 4.07069234  0.07183389 -0.05688262  2.89779738]]
2.2270969959294824


The Frobenius norm itself may not mean much to you. Sometimes it is more meaningful to express this reproduction error as a fraction of the norm of the data matrix itself:

In [14]:
print(np.linalg.norm(M_red-M)/np.linalg.norm(M))

0.16932304913072402
